# Set Up for Project Imports


In [ ]:
import sys
from pathlib import Path
from data.api import UcIrvineAPI, UcIrvineDatasetIDs
import data.wrangling_utils
import duckdb
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    median_absolute_error,
)

In [ ]:
source_directory = Path.cwd()
ROOT = source_directory.parent
data_directory  = ROOT / 'data'
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [ ]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

In [ ]:
pandas.set_option('display.max_colwidth', None)  # show all text in cells
# pandas.set_option("display.max_rows", 100_000)
pandas.options.mode.copy_on_write = True
pandas.set_option('display.float_format', lambda x: '%.2f' % x)

# UcIrvine Data

In [ ]:
uci = UcIrvineAPI.fetch_dataset(repo_id=UcIrvineDatasetIDs.Apartment_For_Rent_Classified.value)
uci_df: pandas.DataFrame = uci.data.original.reset_index()

In [ ]:
uci_df.describe()

In [ ]:
uci_df.shape

# Data Cleaning

In [ ]:
clean_uci_df: pandas.DataFrame = data.wrangling_utils.clean(uci_df)

In [ ]:
clean_uci_df: pandas.DataFrame = data.wrangling_utils.cast(clean_uci_df)
clean_uci_df.info()

In [ ]:
clean_uci_df.rename(columns={'id':'uci_id'}, inplace=True)

In [ ]:
if 'id' not in clean_uci_df.columns:
    clean_uci_df.insert(0, 'id', np.arange(len(clean_uci_df)))

# SQL

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    _ = connection.execute(
    '''
        CREATE TABLE IF NOT EXISTS listings (
            id            BIGINT,
            uci_id        BIGINT,
            category      VARCHAR,
            title         VARCHAR,
            body          VARCHAR,
            amenities     VARCHAR,
            bathrooms     BIGINT,
            bedrooms      BIGINT,
            currency      VARCHAR,
            fee           BOOLEAN,
            has_photo     BOOLEAN,
            pets_allowed  VARCHAR,
            price         DOUBLE,
            price_display DOUBLE,
            price_type    VARCHAR,
            square_feet   DOUBLE,
            address       VARCHAR,
            cityname      VARCHAR,
            state         VARCHAR,
            latitude      DOUBLE,
            longitude     DOUBLE,
            source        VARCHAR,
            time          DOUBLE
        );
        '''
    )
    _ = connection.register("df_view", clean_uci_df)
    rows_before = connection.execute("SELECT COUNT(*) FROM listings").fetchone()[0]
    _ = connection.execute("INSERT INTO listings SELECT * FROM df_view WHERE id NOT IN (SELECT id FROM listings)")
    rows_after = connection.execute("SELECT COUNT(*) FROM listings").fetchone()[0]
    print(f'[Info] Inserted {rows_after - rows_before} records into listings.')


# Exploratory Data Analysis


In [ ]:
df = None
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT category,
    COUNT(*) AS count
    FROM listings
    WHERE
    category IS NOT NULL
    GROUP BY category
    ORDER BY category;
    ''').fetchdf()
df

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT price_type,
    COUNT(*) AS count
    FROM listings
    WHERE
    price_type IS NOT NULL
    GROUP BY price_type
    ORDER BY price_type;
    ''').fetchdf()
df

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT currency,
    COUNT(*) AS count
    FROM listings
    WHERE
    currency IS NOT NULL
    GROUP BY currency
    ORDER BY currency;
    ''').fetchdf()
df

#### Top 5 Listing Sources

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT source,
    COUNT(*) AS count
    FROM listings
    WHERE
    source IS NOT NULL
    GROUP BY source
    ORDER BY count DESC
    LIMIT 5;
    ''').fetchdf()
df

#### Distribution of Count of Listings per State

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT *,
    FROM listings
    WHERE
    state IS NOT NULL AND category IS NOT NULL AND
    price_type  IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly'
    ORDER BY state;
    ''').fetchdf()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 12))
ax = sns.countplot(data=df, x='state', width=0.5)

for container in ax.containers:
    ax.bar_label(container)

plt.xticks(rotation=90)
plt.tight_layout()

There is class imbalance among the states. Each state is not equally represented in the data set. But this is a natural imbalances so we won't resample. Some states have larger than others (e.g., california than vermont).

#### Descriptive Statistics of all listings

In [ ]:
df.describe()

An apartment with 9 bedrooms, a monthly cost of $120,000, and 50,000 square feet? Is this actually an apartment? Let's examine the listings by combinations of bedrooms and bathrooms, and distributions of prices and square footage.. It will give insight on the type of properties in our data.

## Removing Noise

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT bedrooms, bathrooms,
    COUNT(*) AS count
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly'
    GROUP BY bedrooms, bathrooms
    ORDER BY bedrooms, bathrooms;
    ''').fetchdf()
df

It appears studios, townhomes, houses, mansions, or penthouses are also in the data. Some properties don't even have a bedroom? Other properties have 4+ bedrooms and bathrooms? That doesn't make sense for a typical apartment so we will limit our listings to 1-3 bedrooms and 1-3 bathrooms.

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute(f'''
    SELECT bedrooms, bathrooms, price
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly' AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3;
    ''').fetchdf()
df.describe()

Why is 3 bed and 3 bath priced at $33,0000? That seems overpriced even for a luxury apartment. Let's examine the distribution of listing prices for our data.

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT price,
    COUNT(*) AS count
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND price IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly' AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3
    GROUP BY price;
    ''').fetchdf()

In [ ]:
sns.histplot(df['price'], bins=50)
plt.xlim(df['price'].min(), df['price'].max())
plt.show()

The distribution of prices is unimodal, and it is strongly right skewed where most the data is centered around 2,000-3,0000 USD. There are extreme outliers of 10,000-30,000. Maybe luxurious apartments/penthouse are causing this.

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT bedrooms, bathrooms, price
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND price IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly' AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3 AND
    price >= 500 AND price <= 10000;
    ''').fetchdf()
df.describe()

Now let's examine the distribution of prices again.

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT price,
    COUNT(*) AS count
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND price IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly' AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3 AND
    price >= 500 AND price <= 10000
    GROUP BY price;
    ''').fetchdf()
sns.histplot(df['price'], bins=50)
plt.show()

The distribution of the listing prices is still right skewed but we got rid of some properties that weren't representative of a typical apartment in terms of prrice. Now let's examine the distribution of the square footage of the listings since ~5,000 square foot apartment still seems large for a max 3 bedroom and 3 bathroom apartment.

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT square_feet,
    COUNT(*) AS count
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND price IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly' AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3 AND
    price >= 500 AND price <= 10000
    GROUP BY square_feet
    ''').fetchdf()
sns.histplot(df['square_feet'], bins=24)
plt.xlim(df['square_feet'].min(), df['square_feet'].max())
plt.show()

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT bathrooms, bedrooms, price, square_feet
    FROM listings
    WHERE
    category IS NOT NULL AND price_type  IS NOT NULL AND
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND price IS NOT NULL AND
    category LIKE '%apartment%' AND price_type='monthly' AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3 AND
    price >= 500 AND price <= 10000 AND
    square_feet <= 4000;
    ''').fetchdf()
df.describe()

In [ ]:
sns.histplot(df['square_feet'], bins=24)
plt.xlim(df['square_feet'].min(), df['square_feet'].max())
plt.show()

In [ ]:
with duckdb.connect(database= (data_directory / 'apartments.duckdb')) as connection:
    df = connection.execute('''
    SELECT bathrooms, bedrooms, price, square_feet, state, latitude, longitude, cityname
    FROM listings
    WHERE
    bedrooms IS NOT NULL AND bathrooms IS NOT NULL AND
    price IS NOT NULL AND square_feet IS NOT NULL AND
    price_type IS NOT NULL AND state IS NOT NULL AND
    latitude IS NOT NULL AND longitude IS NOT NULL AND
    cityname IS NOT NULL AND category is NOT NULL AND
    bedrooms >= 1 AND bedrooms <= 3 AND
    bathrooms >= 1 AND bathrooms <= 3 AND
    price >= 500 AND price <= 10000 AND
    square_feet <= 4000 AND
    price_type = 'monthly' AND
    category LIKE '%apartment%';
    ''').fetchdf()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
percent_change = (1.0 - (df.shape[0] / uci_df.shape[0])) * 100
print(f"Percent change {percent_change:.3f}%")

## More Skewness Analysis

In [ ]:
df[['price', 'square_feet']].skew()

| Feature       | Skew     | Interpretation            | Action               |
| ------------- |----------|---------------------------|----------------------|
| `price`       | **2.28** | **Strongly right-skewed** | Should transform     |
| `square_feet` | **1.17** | **Strongly right-skewed** | Should transform     |


Most the data is small to large apartments prices around $1,500 - 3,000 but a handful of extremely underpriced/overpriced apartments are skewing the data. So we will perform a log transformation to reduce their influence.

In [ ]:
df['price_log'] = np.log1p(df['price'])
df['square_feet_log'] = np.log1p(df['square_feet'])
df[['price_log', 'square_feet_log']].skew()

| Feature       | Skew              | Interpretation         |
|---------------|-------------------|-------------------------|
| `price`       | **2.28 -> 0.36**  | **Weakly right-skewed** |
| `square_feet` | **1.17 -> -0.21** | **Weakly left-skewed**  |

In [ ]:
sns.histplot(df['price_log'], bins=24)
plt.xlim(df['price_log'].min(), df['price_log'].max())
plt.show()

In [ ]:
sns.histplot(df['square_feet_log'], bins=24)
plt.xlim(df['square_feet_log'].min(), df['square_feet_log'].max())
plt.show()

In [ ]:
grid = sns.pairplot(df[['bathrooms', 'bedrooms', 'price_log', 'square_feet_log']], kind='scatter', corner=True)

There only seems a weak linear relationship between log_price and square_feet_log.

In [ ]:
correlation = df[['bathrooms', 'bedrooms', 'price_log', 'square_feet_log']].corr(numeric_only=True)

In [ ]:
correlation

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Correlation Heatmap", fontsize=14)
sns.heatmap(correlation, annot=True, linewidths=0.5, cmap='mako')
plt.show()

### Variable Types

- Discrete / ordinal: bedrooms, bathrooms — integer counts
- Continuous: price_log, square_feet_log — continuous and normalized
- We used Pearson correlation, the relationships involving discrete counts are approximate linear associations, not strict parametric correlations — but they’re still informative here since the discrete values are ordered and range reasonably (0–9).


| Pair                            | Correlation | Interpretation                                                                                                                                            |
| ------------------------------- |-------------| --------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **bathrooms ↔ bedrooms**        | **0.65**    | Strong positive relationship — homes with more bedrooms usually have more bathrooms.                                                                      |
| **bathrooms ↔ square_feet_log** | **0.68**    | Strong positive correlation — larger houses naturally have more bathrooms.                                                                                |
| **bedrooms ↔ square_feet_log**  | **0.66**    | Same strong pattern — larger homes have more bedrooms.                                                                                                    |
| **price_log ↔ square_feet_log** | **0.36**    | Moderate positive relationship — price generally rises with size, but not perfectly (other factors matter).                                               |
| **price_log ↔ bathrooms**       | **0.31**    | Mild correlation — price increases somewhat with bathroom count, but not linearly.                                                                        |
| **price_log ↔ bedrooms**        | **0.21**    | Weak correlation — price doesn’t increase as predictably with bedroom count, possibly because extra bedrooms add less marginal value than square footage. |


## Skewnesss by State

In [ ]:
state_groups = df.groupby("state")
for state, group in state_groups:
    numeric_df = group[["bathrooms", "bedrooms", "price_log", "square_feet_log"]]
    skew_per_state = numeric_df.skew()
    print(f"{state}\n{skew_per_state}\n")

## Correlation by State

In [ ]:
correlation_by_state = {}
for state, group in state_groups:
    numeric_df = group[["bathrooms", "bedrooms", "price_log", "square_feet_log"]]
    correlation_by_state[state] = numeric_df.corr()

In [ ]:
n_states = len(correlation_by_state.keys())
# Grid size (fits 51)
rows = 8
cols = 7
fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
axes = axes.flatten()
for idx, state in enumerate(correlation_by_state.keys()):
    ax = axes[idx]
    corr = correlation_by_state[state]

    sns.heatmap(
        corr,
        ax=ax,
        cmap="mako",
        square=True,
        xticklabels=True,
        yticklabels=True,
        annot=True,
        cbar=False
    )

    ax.set_title(state, fontsize=14)
    ax.tick_params(axis='x', labelrotation=45, labelsize=10)
    ax.tick_params(axis='y', labelrotation=45, labelsize=10)

# Hide any unused subplot cells
for j in range(len(correlation_by_state.keys()), len(axes)):
    axes[j].axis("off")

fig.suptitle("Correlation Matrices by State", fontsize=18)
plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.99))
plt.show()

# K Means Clustering

In [ ]:
clusterer = KMeans(n_clusters=30, random_state=42)
df['geo_cluster'] = clusterer.fit_predict(df[['latitude', 'longitude']])

In [ ]:
plt.figure(figsize=(8, 8))

ax = sns.scatterplot(
    data=df,
    x="longitude",
    y="latitude",
    hue="geo_cluster",
    palette="tab20",
    s=30
)

handles, labels = ax.get_legend_handles_labels()
plt.legend(
    handles=handles,
    labels=labels,
    title="Cluster",
    ncol=3,
    bbox_to_anchor=(1,1),
    loc="upper left"
)

plt.show()

In [ ]:
X = df[['square_feet_log', 'bedrooms', 'bathrooms', 'state', 'geo_cluster', 'cityname']]
y = df['price_log']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

preprocessor = ColumnTransformer([
    ("square_feet_log_poly", PolynomialFeatures(degree=2, include_bias=False), ['square_feet_log']),
    ('structura_linear', 'passthrough', ['bedrooms', 'bathrooms']),
    ('city_encoded', OneHotEncoder(handle_unknown='ignore'), ['cityname']),
    ('neighborhood_cluster_encoded', OneHotEncoder(handle_unknown='ignore'), ['geo_cluster'])
])

model = Pipeline([
    ('preprocess', preprocessor),
    ('linreg', LinearRegression()),
])

model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# ==========================
# 7. Metrics helper (includes Adjusted R²)
# ==========================
def adjusted_r2(y_true, y_pred, n_features: int) -> float:
    """Compute Adjusted R² given predictions and number of original features."""
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    p = n_features
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


def print_regression_metrics(split_name, y_true, y_pred, n_features: int):
    r2 = r2_score(y_true, y_pred)
    adj_r2 = adjusted_r2(y_true, y_pred, n_features)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    med_ae = median_absolute_error(y_true, y_pred)

    print(f"=== {split_name} Metrics ===")
    print(f"R²:              {r2:.2f}")
    print(f"Adjusted R²:     {adj_r2:.2f}")
    print(f"MSE:             {mse:.2f}")
    print(f"RMSE:            {rmse:.2f}")
    print(f"MAE:             {mae:.2f}")
    print(f"MAPE:            {mape:.2f}")
    print(f"Median Abs Err:  {med_ae:.2f}")
    print()


# ==========================
# 8. Interpret real-world error from log-space metrics
# ==========================
def interpret_log_error_metrics(split_name, y_true, y_pred):
    """
    Interpret log-space error metrics in real price terms.
    Assumes y_true and y_pred are log(price).
    """
    rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))
    mae_log = mean_absolute_error(y_true, y_pred)
    med_ae_log = median_absolute_error(y_true, y_pred)

    # Convert log errors to multiplicative factors
    rmse_factor = np.exp(rmse_log)
    mae_factor = np.exp(mae_log)
    med_factor = np.exp(med_ae_log)

    rmse_pct = (rmse_factor - 1.0) * 100.0
    mae_pct = (mae_factor - 1.0) * 100.0
    med_pct = (med_factor - 1.0) * 100.0

    print(f"=== {split_name} Real-World Error Interpretation ===")
    print(f"Typical error (RMSE):    x {rmse_factor:.3f}  (predictions are typically off by ~{rmse_pct:.1f}% in true price)")
    print(f"Average error (MAE):     x {mae_factor:.3f}  (On average, predictions are off by ~{mae_pct:.1f}% in true price)")
    print(f"Typical core error (Median AE): x {med_factor:.3f}  (For half the homes, predictions are off within ~{med_pct:.1f}% in true price)")
    print()


# number of ORIGINAL features (bathrooms, bedrooms, sqft_log, state)
n_features = model.named_steps["preprocess"].transform(X_train).shape[1]

print_regression_metrics("Train", y_train, y_train_pred, n_features)
print_regression_metrics("Test", y_test, y_test_pred, n_features)

interpret_log_error_metrics("Train", y_train, y_train_pred)
interpret_log_error_metrics("Test", y_test, y_test_pred)


| Metric               | Train Value | Test Value | Interpretation                                                                                                                   |
|----------------------|-------------|------------|----------------------------------------------------------------------------------------------------------------------------------|
| **R²**               | 0.78        | 0.76       | % of variance explained. Model captures ~78% of training variation and ~76% of unseen variation **strong fit, low overfitting**. |
| **MSE**              | 0.04        | 0.04       | Average squared error. Test MSE only slightly higher **good generalization**.                                                    |
| **RMSE**             | 0.20        | 0.20       | Typical prediction error magnitude, ~20% deviation in normalized/log scale.                                                      |
| **MAE**              | 0.15        | 0.15       | Average absolute error, predictions off by ~15%.                                                                                 |
| **MAPE**             | 0.02        | 0.02       | Mean Absolute Percentage Error ~2% if target is scaled, or ~2% error in/log space. Very low **consistent accuracy**.             |
| **Median Abs Error** | 0.11        | 0.12       | Half of predictions are within **11-12% error**. Indicates tight, stable core accuracy.                                          |


In [ ]:
train_resid = y_train - y_train_pred
test_resid  = y_test - y_test_pred

In [ ]:
# Build DataFrames for easy plotting
train_df = pandas.DataFrame({
    "Fitted": y_train_pred,
    "Residual": train_resid,
    "Split": "Train"
})

test_df = pandas.DataFrame({
    "Fitted": y_test_pred,
    "Residual": test_resid,
    "Split": "Test"
})

resid_df = pandas.concat([train_df, test_df], ignore_index=True)

# Plot
plt.figure(figsize=(8, 8))
sns.scatterplot(
    data=resid_df,
    x="Fitted",
    y="Residual",
    hue="Split",
    alpha=0.5
)

plt.axhline(0, linestyle="--", color="black", linewidth=1)
plt.title("Residuals vs Fitted")
plt.xlabel("Fitted")
plt.ylabel("Residual")
plt.tight_layout()
plt.show()

In [ ]:
# DO NOT DELETE MIGHT NEED
# import json
# s = cleaned_uci_df['bathrooms'].explode()
# global_counts = s.value_counts().to_dict()
# global_counts
# print(f'BAD_DATA: {json.dumps(BAD_DATA['cityname'], indent=2)}')  # CHANGE COL
# uci_df["state_full"] = uci_df["state"].str.upper().map(STATE_MAP)
# print(uci_df.shape)
# uci_df.dropna(subset=["state_full"], inplace=True)
# uci_df.shape

# Simple Linear Regression

In [ ]:
lin_regression_residuals = {}
for state, group in state_groups:
    # choose numeric columns only
    X = group[["bathrooms", "bedrooms", "square_feet_log"]]
    y = group[['price_log']]
    model = LinearRegression()
    model.fit(X, y)
    # Predictions and residuals
    y_pred = model.predict(X)
    residuals = y - y_pred
    lin_regression_residuals[state] = y_pred, residuals

In [ ]:
# Grid size (fits 51)
rows = 8
cols = 7
fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
axes = axes.flatten()
for idx, state in enumerate(lin_regression_residuals.keys()):
    ax = axes[idx]
    y_pred, residuals = lin_regression_residuals[state]
    ax.scatter(y_pred, residuals)
    ax.axhline(0, color='red', linestyle='--', linewidth=2)
    ax.set_title(state, fontsize=14)
    ax.set_xlabel('Predicted price_log', fontsize=10)
    ax.set_ylabel('Residuals', fontsize=10)
    ax.set_title(state, fontsize=14)
# Hide any unused subplot cells
for j in range(len(lin_regression_residuals.keys()), len(axes)):
    axes[j].axis("off")
plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.99))
plt.show()

In [ ]:
# Classification Model

Goal: classifiy the listings as overpriced or fair

In [ ]:
X_train_transformed = X_train.copy()
X_train_transformed['pred_price_log'] = y_train_pred
X_train_transformed['pct_overpriced'] = (y_train - X_train_transformed['pred_price_log']) / X_train_transformed['pred_price_log']
X_train_transformed['overpriced'] = (X_train_transformed['pct_overpriced'] > 0.20).astype(int)

numeric_features = ['bedrooms', 'bathrooms', 'square_feet_log', 'pred_price_log', 'pct_overpriced']
categorical_features = ['geo_cluster', 'cityname']

preprocessor_clf = ColumnTransformer([

])


clf_pipeline = Pipeline([
    ('preprocess', preprocessor_clf),
    ('logreg', LogisticRegression())
])